In [8]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime
from sqlalchemy import create_engine
import urllib.parse

CSV Extraction

In [9]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

JSON Extraction

In [10]:
def extract_from_json(file_to_process):
    dataframe= pd.read_json(file_to_process)
    return dataframe

XML Extraction

In [11]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for row in root:
        age = int(row.find("age").text)
        sex = row.find("sex").text
        bmi = float(row.find("bmi").text)
        children = row.find("children").text
        smoker = row.find("smoker").text
        region = row.find("region").text
        charges = float(row.find("charges").text)
        row_data = pd.DataFrame({'age': [age], 'sex': [sex], 'bmi': [bmi], 'children': [children], 'smoker': [smoker], 'region': [region], 'charges': [charges]})
        dataframe = pd.concat([dataframe, row_data], ignore_index=True)
    return dataframe

Extract

In [12]:
def extract():
    columns = ['age', 'sex', 'bmi','children', 'smoker', 'region', 'charges']
    extracted_data = pd.DataFrame(columns=columns)
    
    for csvfile in glob.glob("./datasets/*.csv*"):
        extracted_data = pd.concat([extracted_data, extract_from_csv(csvfile)], ignore_index=True)

    for jsonfile in glob.glob("./datasets/*.json"):
        extracted_data = pd.concat([extracted_data, extract_from_json(jsonfile)], ignore_index=True)

    for xmlfile in glob.glob("./datasets/*.xml"):
        extracted_data = pd.concat([extracted_data, extract_from_xml(xmlfile)], ignore_index=True)

    return extracted_data

Transform

In [1]:
def transform(data):
    data['bmi'] = round(data.bmi, 3)
    data['charges'] = round(data.charges, 3)
    data['smoker'] = data['smoker'].map({'yes': 1, 'no': 0})
    return data

MySQL Connection

In [14]:
host = 'localhost'
user = '' # add your user
password = '' # add your password
database = 'insurance_db'
table_name = 'tbl_insurance'
password = urllib.parse.quote_plus(password)
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

Load

In [15]:
def load(data):
    # if_exists='append' will add data to existing
    # if_exists='replace' will remove the existing data and add new data
    data.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

Log

In [16]:
def log(message):
    timestamp_format = '%Y-%m-%d %H:%M:%S.%f'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./log/logfile.txt", "a") as f:
        f.write(timestamp+', '+message+'\n')

Execution

In [17]:
log("ETL Job Started")
log("Extracting Data")
extracted_data = extract()
log("Data Extraction Completed")
log("Transforming Data")
transformed_data = transform(extracted_data)
log("Data Transformation Completed")
log("Loading Data")
load(transformed_data)
log("Data Loaded")
log("ETL Job Completed")

/tmp/ipykernel_38866/2870120228.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, extract_from_csv(csvfile)], ignore_index=True)
/tmp/ipykernel_38866/1819396679.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, row_data], ignore_index=True)
